In [60]:
import requests
import httplib2
import json
import pandas as pd
from datetime import date 
import xlsxwriter
import csv

In [61]:

def read_csv(path):
    with open(path, encoding='utf-8') as f:
        a = [{str(k):str(v) for k, v in row.items()}
            for row in csv.DictReader(f, skipinitialspace=True)]
    return a

def extract_source(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    source=requests.get(url, headers=headers).text
    content = json.loads(source)
    return content

def extract(url):
    i=0
    content=1
    while content:
        try:
            url = 'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{}?company=2010&evaluated=bool:'.format(str(10*i))
            content = extract_source(url)

            if not content:
                break
            i=i+1
            for item in content["complainResult"]["complains"]["data"]:
                add = {'Title':item['title'],'Complain': item['description'],'Score':item['score'],'DealAgain':item['dealAgain'],'Status':item['status'], 'FirstInteraction':item["firstInteractionDate"]}
                df = df.append(add, ignore_index=True)
        except:
            break

    return df


def splitter(data1):
    data1 = data1.split('T')[0]

    return data1

def splitter2(data1,data2):
    data1 = data1.split('T')[0]
    data2 = data2.split('.')[0]
    return pd.Series([data1,data2])


# Geting Data

In [62]:
def data(company):
    dfava = pd.DataFrame(columns=['Title','Complain','Type','Score','DealAgain','Status', 'Created'])
    dfans = pd.DataFrame(columns=['Title','Complain','Type','Score','DealAgain','Status','Created'])
    dfpen=pd.DataFrame(columns=['Title','Complain','Type','Score','DealAgain','Status', 'Created'])
    i=0
    content=1
    while content:
        try:
            urlans = 'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{}?company={}&evaluated=bool:'.format(str(10*i),company)
            content = extract_source(urlans)

            if not content:
                break
            i=i+1
            for item in content["complainResult"]["complains"]["data"]:
                add = {'Title':item['title'],'Complain': item['description'],'Type':item['otherProblemType'],'Score':item['score'],'DealAgain':item['dealAgain'],'Status':item['status'], 'Created':item['created']}
                dfava = dfava.append(add, ignore_index=True)
        except:
            break
    content=1
    i=0
    while content:
        try:
            urlans = 'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{}?company={}&status=ANSWERED&evaluated=bool:false'.format(str(10*i),company)
            content = extract_source(urlans)

            if not content:
                break
            i=i+1
            for item in content["complainResult"]["complains"]["data"]:
                add = {'Title':item['title'],'Complain': item['description'],'Type':item['otherProblemType'],'Score':item['score'],'DealAgain':item['dealAgain'],'Status':item['status'], 'Created':item['created']}
                dfans = dfans.append(add, ignore_index=True)
        except:
            break
    content=1
    i=0
    while content:
        try:
            urlpen = 'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{}?company={}&status=PENDING&evaluated=bool:false'.format(str(10*i),company)
            content = extract_source(urlpen)

            if not content:
                break
            i=i+1
            for item in content["complainResult"]["complains"]["data"]:
                add = {'Title':item['title'],'Complain': item['description'],'Type':item['otherProblemType'],'Score':item['score'],'DealAgain':item['dealAgain'],'Status':item['status'], 'Created':item['created']}
                dfpen = dfpen.append(add, ignore_index=True)
        except:
            break

    dfpen['Created'] = dfpen.apply(lambda row : splitter(str(row['Created'])), axis = 1)
    dfans['Created'] = dfans.apply(lambda row : splitter(str(row['Created'])), axis = 1)
    dfava[['Created','Score']] = dfava.apply(lambda row : splitter2(str(row['Created']),str(row['Score'])), axis = 1)            
    return dfpen,dfans,dfava


In [63]:
def write_results(dfpen,dfans,dfava,empresa):

    dfpen.to_csv("./results/{}/NaoRespondidas{}.csv".format(empresa,empresa))

    dfava.to_csv("./results/{}/Avaliadas{}.csv".format(empresa,empresa))

    dfans.to_csv("./results/{}/Respondidas{}.csv".format(empresa,empresa))

empresas = read_csv("empresas.csv")

    

In [64]:

for empresa in empresas:
    dfpen,dfans,dfava = data(empresa["Codigo"])
    write_results(dfpen,dfans,dfava,empresa["Empresas"])